# Acid Rain

## Analysis

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import math
import os
import glob
import datetime
import re

In [ ]:
#Read CSV files from a folder

current_directory = str(os.getcwd()) + "\\raw-data\\"
dataframes = []

for file_name in glob.glob(current_directory+'*.csv'):
    current_dataframe = pd.read_csv(file_name, low_memory=False)
    dataframes.append(current_dataframe)
    

In [ ]:
#Check if dataframes have the same columns

if all([set(dataframes[0].columns) == set(df.columns) for df in dataframes]):
    print('Datasets have the same columns')
else:
    print('Datasets do not have the same columns')



In [ ]:
#Find the columns names that have found in some columns but not in others. This way we can create those columns for all the dataframes

columns = []
    
for x in range(0, len(dataframes), 1):
    for y in range(0, len(dataframes), 1):
        for z in range(0, len(dataframes[x].columns), 1):
            #print(str(z) + "||"+ str(len(dataframes[y].columns))+ "||" + str(y))
            if(dataframes[x].columns[z] in dataframes[y].columns):
                pass
            else:
                if (dataframes[x].columns[z] in columns):
                    pass
                else:
                    columns.append(dataframes[x].columns[z])
                

print(columns)

In [ ]:
#Rename the columns those were not found in all the dataframes

for x in range(0, len(dataframes), 1):
    dataframes[x].rename(columns={'Unnamed: 108': 'COMMENT.20', 'Unnamed: 109': 'COMMENT.21', 'Unnamed: 110': 'COMMENT.22'}, inplace=True)
    columns = list(map(lambda x: x.replace('Unnamed: 108', 'COMMENT.20'), columns))
    columns = list(map(lambda x: x.replace('Unnamed: 109', 'COMMENT.21'), columns))
    columns = list(map(lambda x: x.replace('Unnamed: 110', 'COMMENT.22'), columns))

In [ ]:
#Create columns in all the dataframes where they were not already there

for x in range(0, len(dataframes), 1):
    for y in range(0,len(columns),1):
        if columns[y] in dataframes[x].columns:
            pass
        else:
            dataframes[x][columns[y]]=99999

In [ ]:
#Combine all the dataframes into one

dataset_nb_acid_rain_raw = pd.concat(dataframes)
dataset_nb_acid_rain_raw.tail(5)

In [ ]:
#Create a copy of the dataset

nb_acid_rain = dataset_nb_acid_rain_raw.copy()
nb_acid_rain.head(5)

In [ ]:
#Export Combined Dataset to a CSV

nb_acid_rain.to_csv("data/nb_acid_rain_raw.csv", sep=',',index=False,encoding='utf-8-sig')

#Shape of row data
nb_acid_rain.shape


#### Create a column for comments

In [ ]:
#Merge all comment columns to one column with comma separated values

cols = ["COMMENT","COMMENT.1","COMMENT.2","COMMENT.3","COMMENT.4","COMMENT.5","COMMENT.6","COMMENT.7","COMMENT.8","COMMENT.9","COMMENT.10","COMMENT.11","COMMENT.12","COMMENT.13","COMMENT.14","COMMENT.20","COMMENT.21","COMMENT.22"]

nb_acid_rain[cols] = nb_acid_rain[cols].fillna(99999)

nb_acid_rain[cols] = nb_acid_rain[cols].astype(int)
nb_acid_rain[cols] = nb_acid_rain[cols].astype(str)
nb_acid_rain[cols] = nb_acid_rain[cols].replace('99999','')

nb_acid_rain["COMMENTS"] = nb_acid_rain["COMMENT"].copy()+", "+nb_acid_rain["COMMENT.1"].copy()+", "+nb_acid_rain["COMMENT.2"].copy()+", "+nb_acid_rain["COMMENT.3"].copy()+", "+nb_acid_rain["COMMENT.4"].copy()+", "+nb_acid_rain["COMMENT.5"].copy()+", "+nb_acid_rain["COMMENT.6"].copy()+", "+nb_acid_rain["COMMENT.7"].copy()+", "+nb_acid_rain["COMMENT.8"].copy()+", "+nb_acid_rain["COMMENT.9"].copy()+", "+nb_acid_rain["COMMENT.10"].copy()+", "+nb_acid_rain["COMMENT.11"].copy()+", "+nb_acid_rain["COMMENT.12"].copy()+", "+nb_acid_rain["COMMENT.13"].copy()+", "+nb_acid_rain["COMMENT.14"].copy()+", "+nb_acid_rain["COMMENT.20"].copy()+", "+nb_acid_rain["COMMENT.21"].copy()+", "+nb_acid_rain["COMMENT.22"].copy()

nb_acid_rain["COMMENTS"] = nb_acid_rain["COMMENTS"].replace(r'(,\s){2,}', ', ', regex=True)
nb_acid_rain["COMMENTS"] = nb_acid_rain["COMMENTS"].str.strip()
nb_acid_rain["COMMENTS"] = nb_acid_rain["COMMENTS"].str.rstrip(',')

#Delete all the single comment columns
nb_acid_rain = nb_acid_rain.drop(cols, axis=1)

##### Create comment columns with the comment code

In [ ]:
#nb_acid_rain_comments = pd.read_csv("data/nb_acid_rain_comments.csv")

In [ ]:
""" for index, row in nb_acid_rain_comments.iterrows():
    comment_col = "COMMENT_" + str(row["COMMENT_CODE"])
    nb_acid_rain[comment_col] = 0 """


In [ ]:
""" nb_acid_rain = nb_acid_rain.reset_index()

for index, row in nb_acid_rain.iterrows():
    comments = row["COMMENTS"].split(",")
    for i in range(0, len(comments),1):
        if(comments[i] != ""):
            nb_acid_rain.at[index,"COMMENT_" + str(comments[i]).strip()]=1 """

In [ ]:
nb_acid_rain.head(5)

#### Attach station information

In [ ]:
#Import data 

station_information = pd.read_csv("data/nb_acid_rain_station_info.csv")

nb_acid_rain = pd.merge(nb_acid_rain, station_information[["STATION_NAME","LATITUDE","LONGITUDE"]],  how='left', left_on=['STATION_NAME'], right_on = ['STATION_NAME'])

nb_acid_rain.head(5)

#### Drop Empty Columns

In [ ]:
def remove_unit_from_empty(unitVal, unitName):
    if(unitVal == "" or math.isnan(unitVal)):
        return np.nan
    return unitName


In [ ]:
nb_acid_rain["ACDT(Units)"] = nb_acid_rain.apply(lambda x: remove_unit_from_empty(x["ACDT"],x["ACDT(Units)"]),axis=1)
nb_acid_rain["CAD(Units)"] = nb_acid_rain.apply(lambda x: remove_unit_from_empty(x["CAD"],x["CAD(Units)"]),axis=1)
nb_acid_rain["COND(Units)"] = nb_acid_rain.apply(lambda x: remove_unit_from_empty(x["COND"],x["COND(Units)"]),axis=1)
nb_acid_rain["MGD(Units)"] = nb_acid_rain.apply(lambda x: remove_unit_from_empty(x["MGD"],x["MGD(Units)"]),axis=1)
nb_acid_rain["NA(Units)"] = nb_acid_rain.apply(lambda x: remove_unit_from_empty(x["NA"],x["NA(Units)"]),axis=1)
nb_acid_rain["NH3T(Units)"] = nb_acid_rain.apply(lambda x: remove_unit_from_empty(x["NH3T"],x["NH3T(Units)"]),axis=1)
nb_acid_rain["NOX(Units)"] = nb_acid_rain.apply(lambda x: remove_unit_from_empty(x["NOX"],x["NOX(Units)"]),axis=1)
nb_acid_rain["NO2D(Units)"] = nb_acid_rain.apply(lambda x: remove_unit_from_empty(x["NO2D"],x["NO2D(Units)"]),axis=1)
nb_acid_rain["PH(Units)"] = nb_acid_rain.apply(lambda x: remove_unit_from_empty(x["PH"],x["PH(Units)"]),axis=1)
nb_acid_rain["POTASS(Units)"] = nb_acid_rain.apply(lambda x: remove_unit_from_empty(x["POTASS"],x["POTASS(Units)"]),axis=1)
nb_acid_rain["CLIC(Units)"] = nb_acid_rain.apply(lambda x: remove_unit_from_empty(x["CLIC"],x["CLIC(Units)"]),axis=1)
nb_acid_rain["SO4IC(Units)"] = nb_acid_rain.apply(lambda x: remove_unit_from_empty(x["SO4IC"],x["SO4IC(Units)"]),axis=1)
nb_acid_rain["ALKG(Units)"] = nb_acid_rain.apply(lambda x: remove_unit_from_empty(x["ALKG"],x["ALKG(Units)"]),axis=1)
nb_acid_rain["ACDG(Units)"] = nb_acid_rain.apply(lambda x: remove_unit_from_empty(x["ACDG"],x["ACDG(Units)"]),axis=1)
nb_acid_rain["ALKT(Units)"] = nb_acid_rain.apply(lambda x: remove_unit_from_empty(x["ALKT"],x["ALKT(Units)"]),axis=1)
nb_acid_rain["HION(Units)"] = nb_acid_rain.apply(lambda x: remove_unit_from_empty(x["HION"],x["HION(Units)"]),axis=1)
nb_acid_rain["NO3(Units)"] = nb_acid_rain.apply(lambda x: remove_unit_from_empty(x["NO3"],x["NO3(Units)"]),axis=1)
nb_acid_rain["HARD(Units)"] = nb_acid_rain.apply(lambda x: remove_unit_from_empty(x["HARD"],x["HARD(Units)"]),axis=1)
nb_acid_rain["NH4(Units)"] = nb_acid_rain.apply(lambda x: remove_unit_from_empty(x["NH4"],x["NH4(Units)"]),axis=1)
nb_acid_rain["ES04(Units)"] = nb_acid_rain.apply(lambda x: remove_unit_from_empty(x["ESO4"],x["ES04(Units)"]),axis=1)
nb_acid_rain["FE(Units)"] = nb_acid_rain.apply(lambda x: remove_unit_from_empty(x["FE"],x["FE(Units)"]),axis=1)
nb_acid_rain["VANDIUM(Units)"] = nb_acid_rain.apply(lambda x: remove_unit_from_empty(x["VANDIUM"],x["VANDIUM(Units)"]),axis=1)
nb_acid_rain["MN(Units)"] = nb_acid_rain.apply(lambda x: remove_unit_from_empty(x["MN"],x["MN(Units)"]),axis=1)
nb_acid_rain["AL(Units)"] = nb_acid_rain.apply(lambda x: remove_unit_from_empty(x["AL"],x["AL(Units)"]),axis=1)
nb_acid_rain["ZN(Units)"] = nb_acid_rain.apply(lambda x: remove_unit_from_empty(x["ZN"],x["ZN(Units)"]),axis=1)
nb_acid_rain["CD(Units)"] = nb_acid_rain.apply(lambda x: remove_unit_from_empty(x["CD"],x["CD(Units)"]),axis=1)
nb_acid_rain["PB(Units)"] = nb_acid_rain.apply(lambda x: remove_unit_from_empty(x["PB"],x["PB(Units)"]),axis=1)
nb_acid_rain["IB(Units)"] = nb_acid_rain.apply(lambda x: remove_unit_from_empty(x["IB"],x["IB(Units)"]),axis=1)
nb_acid_rain["AS(Units)"] = nb_acid_rain.apply(lambda x: remove_unit_from_empty(x["AS"],x["AS(Units)"]),axis=1)

In [ ]:
#Check for null values
pd.set_option('display.max_rows',None)
nb_acid_rain.isna().sum()

In [ ]:
columns_to_drop = ["FE","FE(Units)","LFE","MN","MN(Units)","LMN","ZN","ZN(Units)","LZN"]

nb_acid_rain = nb_acid_rain.drop(columns_to_drop, axis=1)

In [ ]:
#Rename columns to remove whitespace
nb_acid_rain.rename(columns={'SAMPLE_WT (g)': 'SAMPLE_WT_(g)', 'PRECIP (mm)': 'PRECIP_(mm)', 'EFFIC (%)': 'EFFIC_(%)', 'LAB_WT (g)': 'LAB_WT_(g)'}, inplace=True)

In [ ]:
#Rearrange columns
#cols = ["STATION_NAME","STATION_ID","LATITUDE","LONGITUDE","LAB_NO","FIELD_NO","FROM_DATE","TO_DATE","WEEK_NO","YEAR_NO","SAMPLE_WT_(g)","PRECIP_(mm)","CALC_PRECIP","EFFIC_(%)","SUBSTAND","LSUBSTAND","LAB_WT_(g)","ACDT","ACDT(Units)","LACDT","CAD","CAD(Units)","LCAD","COND","COND(Units)","LCOND","MGD","MGD(Units)","LMGD","NA","NA(Units)","LNA","NH3T","NH3T(Units)","LNH3T","NOX","NOX(Units)","LNOX","NO2D","NO2D(Units)","LNO2D","PH","PH(Units)","LPH","POTASS","POTASS(Units)","LPOTASS","CLIC","CLIC(Units)","LCLIC","SO4IC","SO4IC(Units)","LSO4IC","ALKG","ALKG(Units)","LALKG","ACDG","ACDG(Units)","LACDG","ALKT","ALKT(Units)","LALKT","HION","HION(Units)","NO3","NO3(Units)","LNO3","HARD","HARD(Units)","LHARD","NH4","NH4(Units)","LNH4","ESO4","ES04(Units)","LESO4","VANDIUM","VANDIUM(Units)","LVANDIUM","AL","AL(Units)","LAL","CD","CD(Units)","LCD","PB","PB(Units)","LPB","IB","IB(Units)","AS","AS(Units)","LAS","ONE_EVENT","SMPL_MISS","COMMENTS","COMMENT_110","COMMENT_111","COMMENT_112","COMMENT_113","COMMENT_114","COMMENT_115","COMMENT_116","COMMENT_117","COMMENT_118","COMMENT_119","COMMENT_120","COMMENT_121","COMMENT_122","COMMENT_123","COMMENT_124","COMMENT_125","COMMENT_126","COMMENT_127","COMMENT_128","COMMENT_129","COMMENT_130","COMMENT_131","COMMENT_132","COMMENT_133","COMMENT_134","COMMENT_135","COMMENT_136","COMMENT_137","COMMENT_138","COMMENT_139","COMMENT_140","COMMENT_141","COMMENT_142","COMMENT_143","COMMENT_144","COMMENT_145","COMMENT_146","COMMENT_147","COMMENT_148","COMMENT_149","COMMENT_150","COMMENT_151","COMMENT_152","COMMENT_153","COMMENT_154","COMMENT_155","COMMENT_156","COMMENT_157","COMMENT_158","COMMENT_159","COMMENT_160","COMMENT_161","COMMENT_162","COMMENT_163","COMMENT_164","COMMENT_165","COMMENT_166","COMMENT_167","COMMENT_168","COMMENT_169","COMMENT_170","COMMENT_171","COMMENT_172","COMMENT_173","COMMENT_174","COMMENT_175","COMMENT_176","COMMENT_177","COMMENT_178","COMMENT_179","COMMENT_180","COMMENT_181","COMMENT_182","COMMENT_183","COMMENT_184","COMMENT_185","COMMENT_186","COMMENT_187","COMMENT_188","COMMENT_189","COMMENT_190","COMMENT_191","COMMENT_192","COMMENT_193","COMMENT_194","COMMENT_195","COMMENT_196","COMMENT_197","COMMENT_198","COMMENT_199","COMMENT_253","COMMENT_285","COMMENT_26"]
cols = ["STATION_NAME","STATION_ID","LATITUDE","LONGITUDE","LAB_NO","FIELD_NO","FROM_DATE","TO_DATE","WEEK_NO","YEAR_NO","SAMPLE_WT_(g)","PRECIP_(mm)","CALC_PRECIP","EFFIC_(%)","SUBSTAND","LSUBSTAND","LAB_WT_(g)","ACDT","ACDT(Units)","LACDT","CAD","CAD(Units)","LCAD","COND","COND(Units)","LCOND","MGD","MGD(Units)","LMGD","NA","NA(Units)","LNA","NH3T","NH3T(Units)","LNH3T","NOX","NOX(Units)","LNOX","NO2D","NO2D(Units)","LNO2D","PH","PH(Units)","LPH","POTASS","POTASS(Units)","LPOTASS","CLIC","CLIC(Units)","LCLIC","SO4IC","SO4IC(Units)","LSO4IC","ALKG","ALKG(Units)","LALKG","ACDG","ACDG(Units)","LACDG","ALKT","ALKT(Units)","LALKT","HION","HION(Units)","NO3","NO3(Units)","LNO3","HARD","HARD(Units)","LHARD","NH4","NH4(Units)","LNH4","ESO4","ES04(Units)","LESO4","VANDIUM","VANDIUM(Units)","LVANDIUM","AL","AL(Units)","LAL","CD","CD(Units)","LCD","PB","PB(Units)","LPB","IB","IB(Units)","AS","AS(Units)","LAS","ONE_EVENT","SMPL_MISS","COMMENTS"]

nb_acid_rain = nb_acid_rain[cols]

In [ ]:
#Display unique values

def unique_values__or_count(listOfColumns,options,dataset):
    for x in range(0, len(listOfColumns), 1):
        if(options=="unique"):
            unique_values_str = dataset[listOfColumns[x]].unique()
            print("unique_values " + listOfColumns[x])
            print(unique_values_str)
            print("------------------------")
        if(options=="count"):
            values_distribution = dataset[listOfColumns[x]].value_counts()
            print("-----------"+listOfColumns[x] +"------------")
            print(values_distribution)
            print("-----------------------")

#### Unit Columns processing

In [ ]:
cols = ["ACDT(Units)","CAD(Units)","COND(Units)","MGD(Units)","NA(Units)","NH3T(Units)","NOX(Units)","NO2D(Units)","PH(Units)","POTASS(Units)","CLIC(Units)","SO4IC(Units)","ALKG(Units)","ACDG(Units)","ALKT(Units)","HION(Units)","NO3(Units)","HARD(Units)","NH4(Units)","ES04(Units)","VANDIUM(Units)","AL(Units)","CD(Units)","PB(Units)","IB(Units)","AS(Units)"]

nb_acid_rain[cols] = nb_acid_rain[cols].fillna("")

In [ ]:
def get_unit_val(unit_val):
    unit_val = list(filter(None, unit_val))
    return ', '.join(unit_val)

In [ ]:
units_data = [["ACDT", get_unit_val(nb_acid_rain["ACDT(Units)"].unique())],
 ["CAD", get_unit_val(nb_acid_rain["CAD(Units)"].unique())],
 ["COND", get_unit_val(nb_acid_rain["COND(Units)"].unique())],
 ["MGD", get_unit_val(nb_acid_rain["MGD(Units)"].unique())],
 ["NA", get_unit_val(nb_acid_rain["NA(Units)"].unique())],
 ["NH3T", get_unit_val(nb_acid_rain["NH3T(Units)"].unique())],
 ["NOX", get_unit_val(nb_acid_rain["NOX(Units)"].unique())],
 ["NO2D", get_unit_val(nb_acid_rain["NO2D(Units)"].unique())],
 ["PH", get_unit_val(nb_acid_rain["PH(Units)"].unique())],
 ["POTASS", get_unit_val(nb_acid_rain["POTASS(Units)"].unique())],
 ["CLIC", get_unit_val(nb_acid_rain["CLIC(Units)"].unique())],
 ["SO4IC", get_unit_val(nb_acid_rain["SO4IC(Units)"].unique())],
 ["ALKG", get_unit_val(nb_acid_rain["ALKG(Units)"].unique())],
 ["ACDG", get_unit_val(nb_acid_rain["ACDG(Units)"].unique())],
 ["ALKT", get_unit_val(nb_acid_rain["ALKT(Units)"].unique())],
 ["HION", get_unit_val(nb_acid_rain["HION(Units)"].unique())],
 ["NO3", get_unit_val(nb_acid_rain["NO3(Units)"].unique())],
 ["HARD", get_unit_val(nb_acid_rain["HARD(Units)"].unique())],
 ["NH4", get_unit_val(nb_acid_rain["NH4(Units)"].unique())],
 ["ESO4", get_unit_val(nb_acid_rain["ES04(Units)"].unique())],
 ["VANDIUM", get_unit_val(nb_acid_rain["VANDIUM(Units)"].unique())],
 ["AL", get_unit_val(nb_acid_rain["AL(Units)"].unique())],
 ["CD", get_unit_val(nb_acid_rain["CD(Units)"].unique())],
 ["PB", get_unit_val(nb_acid_rain["PB(Units)"].unique())],
 ["IB", get_unit_val(nb_acid_rain["IB(Units)"].unique())],
 ["AS", get_unit_val(nb_acid_rain["AS(Units)"].unique())]]

acid_rain_units = pd.DataFrame(units_data, columns=['UNIT_NAME', 'UNITS'])
acid_rain_units.head(15)

#Export units Dataset to a CSV
acid_rain_units.to_csv("data/nb_acid_rain_units.csv", sep=',',index=False,encoding='utf-8-sig')

In [ ]:
unique_values__or_count(["ACDT(Units)", "CAD(Units)", "COND(Units)", "MGD(Units)", "NA(Units)", "NH3T(Units)", "NOX(Units)", "NO2D(Units)", "PH(Units)", "POTASS(Units)", "CLIC(Units)", "SO4IC(Units)", "ALKG(Units)", "ACDG(Units)", "ALKT(Units)", "HION(Units)", "NO3(Units)", "HARD(Units)", "NH4(Units)", "ES04(Units)", "VANDIUM(Units)", "AL(Units)",  "CD(Units)", "PB(Units)", "IB(Units)", "AS(Units)"],"count",nb_acid_rain)

In [ ]:
#Delete all the unit columns

#cols = ["ACDT(Units)", "CAD(Units)", "COND(Units)", "MGD(Units)", "NA(Units)", "NH3T(Units)", "NOX(Units)", "NO2D(Units)", "PH(Units)", "POTASS(Units)", "CLIC(Units)", "SO4IC(Units)", "ALKG(Units)", "ACDG(Units)", "ALKT(Units)", "HION(Units)", "NO3(Units)", "HARD(Units)", "NH4(Units)", "ES04(Units)", "FE(Units)", "VANDIUM(Units)", "MN(Units)", "AL(Units)", "ZN(Units)", "CD(Units)", "PB(Units)", "IB(Units)", "AS(Units)"]

#nb_acid_rain = nb_acid_rain.drop(cols, axis=1)

In [ ]:
nb_acid_rain.isna().sum()

#### Check for duplicates

In [ ]:
nb_acid_rain[nb_acid_rain[['STATION_NAME', 'TO_DATE']].duplicated()]

#### Generate final CSV file

In [ ]:
#Sort the data by date
nb_acid_rain.sort_values(by='TO_DATE', inplace = True)

In [ ]:
#Export Combined Dataset to a CSV

nb_acid_rain.to_csv("data/nb_acid_rain.csv", sep=',',index=False,encoding='utf-8-sig')


### Exploratory Data Analysis

In [ ]:
nb_acid_rain.shape

In [ ]:
nb_acid_rain.info(verbose=True, show_counts=True)

In [ ]:
unique_values__or_count(['STATION_NAME'],"count",nb_acid_rain)

In [ ]:
pd.reset_option('display.max_rows')

### Visualizations

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:

#This is just an example of a line graph, visualization can be better
def linechart_of_categories(dataset,group_by,time_column,value_column):

    dataset = dataset.set_index(time_column)
    dataset = dataset.groupby([group_by])
    # extract keys from groups
    keys = dataset.groups.keys()

    totalCols=3
    totalRows=math.ceil(len(dataset)/totalCols)
    widthForEachGraph=3
    fig = plt.figure(figsize=((totalCols+3)*3,(totalRows+1)*5))
    plt.subplots_adjust(wspace=0.2, hspace=0.6)

    for index, x in enumerate(keys):
        globals()[f"ax_count_plots_{index}"] = fig.add_subplot(totalRows,totalCols,(index+1))
        globals()[f"ax_count_plots_{index}"].set_title(x.upper())
        
        dataset[value_column].get_group(x).plot()
        
        globals()[f"ax_count_plots_{index}"].set(xlabel=None)
        globals()[f"ax_count_plots_{index}"].tick_params(axis='x', labelrotation = 90)
        
    plt.show()


In [ ]:
linechart_of_categories(nb_acid_rain,'STATION_NAME','TO_DATE','PH')

### Data Validation

In [ ]:
#Import data 

nb_acid_rain_skim = pd.read_csv("data/nb_acid_rain.csv")

In [ ]:
nb_acid_rain_validate_with = nb_acid_rain_skim[["STATION_NAME", "STATION_ID", "LAB_NO", "FIELD_NO", "FROM_DATE", "TO_DATE","PH","ACDT"]].copy()

In [ ]:
#Import data 

nb_acid_rain_original = pd.read_csv("raw-data/NB-2021.csv")

nb_acid_rain_validate = nb_acid_rain_original[["STATION_NAME", "STATION_ID", "LAB_NO", "FIELD_NO", "FROM_DATE", "TO_DATE","PH","ACDT"]].copy()

#If you want to validate only a sample

#nb_acid_rain_validate = nb_acid_rain_validate.sample(frac=0.1, random_state=56)


In [ ]:
nb_acid_rain_validate_results = pd.merge(nb_acid_rain_validate_with, nb_acid_rain_validate, on=["STATION_NAME", "STATION_ID", "LAB_NO", "FIELD_NO", "FROM_DATE", "TO_DATE","PH","ACDT"], how='right', indicator='Exist')
nb_acid_rain_validate_results['Exist'] = np.where(nb_acid_rain_validate_results.Exist == 'both', True, False)

In [ ]:
unique_values__or_count(['Exist'],"count",nb_acid_rain_validate_results)

In [ ]:
list_of_missing_rows = nb_acid_rain_validate_results[(nb_acid_rain_validate_results['Exist'] == False)].copy()

list_of_missing_rows.head(5)